<a href="https://colab.research.google.com/github/nthkoldigital/contact-update/blob/main/GoogleContactsSync_WithServiceAccount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Contacts Sync from CSV
#
# This notebook syncs contact information from a CSV file with Google Contacts
# It identifies contacts by Organization Name, and updates or creates them as needed

# These libraries should already be installed in Colab
# If not, uncomment the following line:
# !pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pandas

import pandas as pd
import os
import pickle
import time  # Added for rate limiting
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.auth.exceptions import RefreshError
from google.colab import auth, drive

# Define the scopes needed for Google People API - moved to global scope
SCOPES = ['https://www.googleapis.com/auth/contacts']

def authenticate_google_api():
    """Authenticate with Google API using service account credentials or OAuth"""
    from google.oauth2 import service_account

    # Define the scopes needed for Google People API
    SCOPES = ['https://www.googleapis.com/auth/contacts']

    # Check if service account credentials file exists
    if os.path.exists('service-account.json'):
        try:
            # For domain-wide delegation with service account
            SERVICE_ACCOUNT_FILE = 'service-account.json'

            # The email of the user you want to impersonate
            # This should be the admin or user that has access to the contacts
            print("Enter the email address of the user to impersonate (usually admin):")
            user_email = input()

            credentials = service_account.Credentials.from_service_account_file(
                SERVICE_ACCOUNT_FILE,
                scopes=SCOPES,
                subject=user_email  # User to impersonate
            )

            print("Authentication successful with service account!")
            return credentials
        except Exception as e:
            print(f"Error during service account authentication: {e}")
            print("Falling back to OAuth flow...")
    else:
        print("Service account credentials not found (service-account.json).")
        print("Falling back to OAuth flow...")

    # If service account fails or doesn't exist, fall back to OAuth flow
    from google_auth_oauthlib.flow import InstalledAppFlow
    import json

    # Use credentials.json to create a flow for proper scopes
    if os.path.exists('credentials.json'):
        try:
            # Load client configuration
            with open('credentials.json', 'r') as f:
                client_config = json.load(f)

            # Modify client config to use redirect URI that works in Colab
            client_config['installed']['redirect_uris'] = ['urn:ietf:wg:oauth:2.0:oob']

            # Create flow with the modified client config
            flow = InstalledAppFlow.from_client_config(
                client_config,
                scopes=SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )

            # Run the authorization flow
            auth_url, _ = flow.authorization_url(prompt='consent')
            print(f"\n🔑 Please visit this URL to authorize access to your Google Contacts:\n{auth_url}\n")
            code = input("Enter the authorization code: ")
            flow.fetch_token(code=code)

            # Get credentials
            creds = flow.credentials

            # Save credentials for next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

            print("Authentication successful with proper Contacts API scope!")
            return creds
        except Exception as e:
            print(f"Error during OAuth flow: {e}")

    # If all authentication methods fail, try Colab's default auth as last resort
    try:
        from google.colab import auth
        print("\n⚠️ Falling back to Colab's default authentication (THIS MAY NOT WORK FOR CONTACTS API)")
        print("This might not provide the necessary permissions for Contacts API.")
        auth.authenticate_user()

        from google.auth import default
        creds, _ = default()
        return creds
    except Exception as e:
        print(f"Error with Colab default authentication: {e}")

    print("\n❌ Authentication failed. Unable to proceed.")
    return None

def read_csv_file(file_path):
    """Read CSV file and return pandas DataFrame with only valid records"""
    try:
        # Use dtype parameter to ensure phone numbers are read as strings
        df = pd.read_csv(file_path, dtype={'Phone 1 - Value': str})

        # Clean up data - handle missing values
        df = df.fillna('')

        # Filter out rows without Organization Name (our unique identifier)
        # Based on analysis, most rows don't have organization names
        df = df[df['Organization Name'].str.strip() != '']

        print(f"Found {len(df)} valid contacts with Organization Name")

        return df
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

def get_organization_name(person_resource):
    """Extract organization name from a person resource"""
    organizations = person_resource.get('organizations', [])
    if organizations:
        return organizations[0].get('name', 'Unknown Organization')
    return 'Unknown Organization'

def find_contact_by_organization(service, organization_name, request_counter=None):
    """
    Find a contact by organization name
    Returns the contact resource if found, None otherwise
    Implements rate limiting and retries on quota exceeded errors
    """
    if not organization_name or organization_name == '':
        return None

    # Initialize request counter if not provided
    if request_counter is None:
        request_counter = {'count': 0, 'last_reset': time.time()}

    # Check if we need to reset the counter (more than a minute has passed)
    current_time = time.time()
    if current_time - request_counter['last_reset'] > 60:
        request_counter['count'] = 0
        request_counter['last_reset'] = current_time

    # Check if we're approaching the rate limit (80 requests out of 90 per minute)
    if request_counter['count'] >= 80:
        wait_time = 60 - (current_time - request_counter['last_reset']) + 5  # Add 5 seconds buffer
        if wait_time > 0:
            print(f"⏳ Approaching rate limit. Waiting {wait_time:.1f} seconds before continuing...")
            time.sleep(wait_time)
            request_counter['count'] = 0
            request_counter['last_reset'] = time.time()

    max_retries = 3
    retry_count = 0
    backoff_time = 2  # Start with 2 seconds

    while retry_count <= max_retries:
        try:
            # Increment request counter
            request_counter['count'] += 1

            # Search for contacts with the specified organization name
            results = service.people().connections().list(
                resourceName='people/me',
                pageSize=1000,
                personFields='names,organizations,emailAddresses,phoneNumbers,metadata'
            ).execute()

            connections = results.get('connections', [])

            for person in connections:
                organizations = person.get('organizations', [])
                for org in organizations:
                    if org.get('name', '').lower() == organization_name.lower():
                        return person

            return None

        except HttpError as e:
            error_message = str(e)

            # Handle quota exceeded errors
            if "429" in error_message or "RATE_LIMIT_EXCEEDED" in error_message:
                retry_count += 1

                if retry_count <= max_retries:
                    # Exponential backoff
                    wait_time = backoff_time * (2 ** (retry_count - 1))
                    print(f"⏳ Rate limit exceeded. Waiting {wait_time} seconds before retry {retry_count}/{max_retries}...")
                    time.sleep(wait_time)

                    # Reset counter after waiting
                    request_counter['count'] = 0
                    request_counter['last_reset'] = time.time()
                else:
                    print(f"❌ Error finding contact by organization: Maximum retries exceeded. Rate limit issue persists.")
                    return None
            else:
                print(f"❌ Error finding contact by organization: {e}")
                return None

    return None

def update_contact(service, person_resource, first_name, middle_name, last_name, email, phone, request_counter=None):
    """Update an existing contact with all fields including names"""
    try:
        # Initialize request counter if not provided
        if request_counter is None:
            request_counter = {'count': 0, 'last_reset': time.time()}

        # Check if we need to reset the counter (more than a minute has passed)
        current_time = time.time()
        if current_time - request_counter['last_reset'] > 60:
            request_counter['count'] = 0
            request_counter['last_reset'] = current_time

        # Check if we're approaching the rate limit
        if request_counter['count'] >= 80:
            wait_time = 60 - (current_time - request_counter['last_reset']) + 5  # Add 5 seconds buffer
            if wait_time > 0:
                print(f"⏳ Approaching rate limit. Waiting {wait_time:.1f} seconds before continuing...")
                time.sleep(wait_time)
                request_counter['count'] = 0
                request_counter['last_reset'] = time.time()

        person_id = person_resource['resourceName']
        org_name = get_organization_name(person_resource)
        changes_made = []

        # Check for name changes
        current_names = person_resource.get('names', [])
        current_first_name = current_names[0].get('givenName', '') if current_names else ''
        current_middle_name = current_names[0].get('middleName', '') if current_names else ''
        current_last_name = current_names[0].get('familyName', '') if current_names else ''

        name_changed = True
        if first_name and first_name != current_first_name:
            changes_made.append(f"first name: '{current_first_name}' → '{first_name}'")
            name_changed = True

        if middle_name and middle_name != current_middle_name:
            changes_made.append(f"middle name: '{current_middle_name}' → '{middle_name}'")
            name_changed = True

        if last_name and last_name != current_last_name:
            changes_made.append(f"last name: '{current_last_name}' → '{last_name}'")
            name_changed = True

        # Handle email update
        email_changed = True
        if email and email != '':
            current_emails = person_resource.get('emailAddresses', [])
            current_email = current_emails[0].get('value', '') if current_emails else ''

            if current_email != email:
                changes_made.append(f"email: '{current_email}' → '{email}'")
                email_changed = True

        # Handle phone update
        phone_changed = True
        if phone and phone != '':
            current_phones = person_resource.get('phoneNumbers', [])
            current_phone = current_phones[0].get('value', '') if current_phones else ''

            if current_phone != phone:
                changes_made.append(f"phone: '{current_phone}' → '{phone}'")
                phone_changed = True

        # If no changes needed, return early
        #if not changes_made:
        #    print(f"📝 {org_name}: No changes needed")
        #    return None

        # Prepare update fields and person object
        update_person = {
            # Include the etag to satisfy Google's API requirements
            'etag': person_resource.get('etag', '')
        }

        # Handle name updates
        if name_changed:
            update_person['names'] = [
                {
                    'givenName': first_name if first_name else current_first_name,
                    'middleName': middle_name if middle_name else current_middle_name,
                    'familyName': last_name if last_name else current_last_name
                }
            ]

        # Handle email update
        if email_changed:
            current_emails = person_resource.get('emailAddresses', [])

            # If there are existing emails, update the first one
            if current_emails:
                update_person['emailAddresses'] = [
                    {
                        'value': email,
                        'type': 'work',
                        'metadata': {
                            'primary': True
                        }
                    }
                ]
            else:
                # If no existing emails, add a new one
                update_person['emailAddresses'] = [
                    {
                        'value': email,
                        'type': 'work'
                    }
                ]

        # Handle phone update
        if phone_changed:
            current_phones = person_resource.get('phoneNumbers', [])

            # If there are existing phones, update the first one
            if current_phones:
                update_person['phoneNumbers'] = [
                    {
                        'value': phone,
                        'type': 'work',
                        'metadata': {
                            'primary': True
                        }
                    }
                ]
            else:
                # If no existing phones, add a new one
                update_person['phoneNumbers'] = [
                    {
                        'value': phone,
                        'type': 'work'
                    }
                ]

        # Only perform update if there's data to update
        if len(update_person) > 1:  # More than just etag
            # Define which fields to update
            updatePersonFields = []
            if 'names' in update_person:
                updatePersonFields.append('names')
            if 'emailAddresses' in update_person:
                updatePersonFields.append('emailAddresses')
            if 'phoneNumbers' in update_person:
                updatePersonFields.append('phoneNumbers')

            max_retries = 3
            retry_count = 0
            backoff_time = 2  # Start with 2 seconds

            while retry_count <= max_retries:
                try:
                    # Increment request counter
                    request_counter['count'] += 1

                    # Make the update request
                    result = service.people().updateContact(
                        resourceName=person_id,
                        updatePersonFields=','.join(updatePersonFields),
                        body=update_person
                    ).execute()

                    # Log what was updated
                    print(f"✅ {org_name}: Updated {', '.join(changes_made)}")

                    return result

                except HttpError as e:
                    error_message = str(e)

                    # Handle quota exceeded errors
                    if "429" in error_message or "RATE_LIMIT_EXCEEDED" in error_message:
                        retry_count += 1

                        if retry_count <= max_retries:
                            # Exponential backoff
                            wait_time = backoff_time * (2 ** (retry_count - 1))
                            print(f"⏳ Rate limit exceeded. Waiting {wait_time} seconds before retry {retry_count}/{max_retries}...")
                            time.sleep(wait_time)

                            # Reset counter after waiting
                            request_counter['count'] = 0
                            request_counter['last_reset'] = time.time()
                        else:
                            print(f"❌ Error updating contact: Maximum retries exceeded. Rate limit issue persists.")
                            return None
                    else:
                        print(f"❌ Error updating contact: {e}")
                        return None

        return None
    except HttpError as e:
        print(f"❌ Error updating contact: {e}")
        return None

def create_contact(service, first_name, middle_name, last_name, organization_name, organization_title, email, phone, request_counter=None):
    """Create a new contact with the given information"""
    try:
        # Initialize request counter if not provided
        if request_counter is None:
            request_counter = {'count': 0, 'last_reset': time.time()}

        # Check if we need to reset the counter (more than a minute has passed)
        current_time = time.time()
        if current_time - request_counter['last_reset'] > 60:
            request_counter['count'] = 0
            request_counter['last_reset'] = current_time

        # Check if we're approaching the rate limit
        if request_counter['count'] >= 80:
            wait_time = 60 - (current_time - request_counter['last_reset']) + 5  # Add 5 seconds buffer
            if wait_time > 0:
                print(f"⏳ Approaching rate limit. Waiting {wait_time:.1f} seconds before continuing...")
                time.sleep(wait_time)
                request_counter['count'] = 0
                request_counter['last_reset'] = time.time()

        contact_body = {
            'names': [
                {
                    'givenName': first_name,
                    'middleName': middle_name,
                    'familyName': last_name
                }
            ],
            'organizations': [
                {
                    'name': organization_name,
                    'title': organization_title
                }
            ]
        }

        field_info = []

        # Add email if provided
        if email and email != '':
            contact_body['emailAddresses'] = [
                {
                    'value': email,
                    'type': 'work'
                }
            ]
            field_info.append(f"email: '{email}'")

        # Add phone if provided
        if phone and phone != '':
            contact_body['phoneNumbers'] = [
                {
                    'value': phone,
                    'type': 'work'
                }
            ]
            field_info.append(f"phone: '{phone}'")

        max_retries = 3
        retry_count = 0
        backoff_time = 2  # Start with 2 seconds

        while retry_count <= max_retries:
            try:
                # Increment request counter
                request_counter['count'] += 1

                result = service.people().createContact(
                    body=contact_body
                ).execute()

                # Log the new contact details
                name_parts = []
                if first_name:
                    name_parts.append(f"first: '{first_name}'")
                if middle_name:
                    name_parts.append(f"middle: '{middle_name}'")
                if last_name:
                    name_parts.append(f"last: '{last_name}'")

                if name_parts:
                    field_info.append(f"name ({', '.join(name_parts)})")
                if organization_title:
                    field_info.append(f"title: '{organization_title}'")

                print(f"➕ {organization_name}: Added new contact with {', '.join(field_info)}")

                return result

            except HttpError as e:
                error_message = str(e)

                # Handle quota exceeded errors
                if "429" in error_message or "RATE_LIMIT_EXCEEDED" in error_message:
                    retry_count += 1

                    if retry_count <= max_retries:
                        # Exponential backoff
                        wait_time = backoff_time * (2 ** (retry_count - 1))
                        print(f"⏳ Rate limit exceeded. Waiting {wait_time} seconds before retry {retry_count}/{max_retries}...")
                        time.sleep(wait_time)

                        # Reset counter after waiting
                        request_counter['count'] = 0
                        request_counter['last_reset'] = time.time()
                    else:
                        print(f"❌ {organization_name}: Error creating contact: Maximum retries exceeded. Rate limit issue persists.")
                        return None
                else:
                    print(f"❌ {organization_name}: Error creating contact: {e}")
                    return None

    except Exception as e:
        print(f"❌ {organization_name}: Error creating contact: {e}")
        return None

    return None

def process_contacts_from_csv(csv_file_path):
    """
    Main function to process contacts from a CSV file
    - If contact with matching organization name exists, update it
    - Otherwise, create a new contact
    """
    # Read CSV file
    df = read_csv_file(csv_file_path)
    if df is None:
        print("Failed to read CSV file.")
        return

    if len(df) == 0:
        print("No valid contacts found with Organization Name in the CSV file.")
        return

    # Authenticate with Google API
    try:
        print("Authenticating with Google API...")
        creds = authenticate_google_api()

        # Check if authentication was successful
        if creds is None:
            print("Authentication failed. Unable to proceed.")
            return

        service = build('people', 'v1', credentials=creds)

        # Verify the authentication worked for contacts API
        try:
            # Test with a simple request
            service.people().connections().list(
                resourceName='people/me',
                pageSize=1,
                personFields='names'
            ).execute()
            print("API access verified successfully!")
        except HttpError as e:
            if "PERMISSION_DENIED" in str(e) or "ACCESS_TOKEN_SCOPE_INSUFFICIENT" in str(e):
                print("\n❌ ERROR: Your Google account doesn't have permission to access Contacts.")
                print("This is likely because the OAuth scope for contacts wasn't properly authorized.")
                print("\nTo fix this issue:")
                print("1. Make sure the People API is enabled in your Google Cloud Project")
                print("2. Verify your credentials.json file is from a project with People API enabled")
                print("3. Delete token.pickle if it exists (to force a new authentication)")
                print("4. You may need to run this notebook in a new Colab instance")
                return
            else:
                print(f"API testing error: {e}")
                print("Continuing anyway - this might or might not work...")
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        print("Please make sure you've granted the necessary permissions.")
        return

    # Set up counters for reporting
    contacts_updated = 0
    contacts_created = 0
    contacts_processed = 0
    contacts_skipped = 0
    contacts_failed = 0

    # Create a shared request counter for rate limiting
    request_counter = {'count': 0, 'last_reset': time.time()}

    # Print header for the sync process
    print("\n" + "="*60)
    print(" 🔄 GOOGLE CONTACTS SYNC PROCESS STARTED")
    print("="*60)

    # Process each row in the CSV
    for index, row in df.iterrows():
        try:
            # Extract necessary fields
            first_name = str(row.get('First Name', '')).strip()
            middle_name = str(row.get('Middle Name', '')).strip()
            last_name = str(row.get('Last Name', '')).strip()
            organization_name = str(row.get('Organization Name', '')).strip()
            organization_title = str(row.get('Organization Title', '')).strip()
            email = str(row.get('E-mail 1 - Value', '')).strip()

            # Ensure phone is treated as string to prevent floating point conversion
            phone_value = row.get('Phone 1 - Value', '')
            phone = str(phone_value).strip() if phone_value else ''
            # Remove .0 suffix if it was added during CSV parsing
            if phone.endswith('.0') and phone[:-2].isdigit():
                phone = phone[:-2]

            # Skip rows without organization name (our unique identifier)
            if not organization_name:
                print(f"⏩ Row {index+2}: Skipping - No organization name")
                contacts_skipped += 1
                continue

            contacts_processed += 1

            # Check if contact exists
            existing_contact = find_contact_by_organization(service, organization_name, request_counter)

            if existing_contact:
                # Update existing contact with all fields
                result = update_contact(
                    service=service,
                    person_resource=existing_contact,
                    first_name=first_name,
                    middle_name=middle_name,
                    last_name=last_name,
                    email=email,
                    phone=phone,
                    request_counter=request_counter
                )
                if result:
                    contacts_updated += 1
            else:
                # Create new contact with all fields
                result = create_contact(
                    service=service,
                    first_name=first_name,
                    middle_name=middle_name,
                    last_name=last_name,
                    organization_name=organization_name,
                    organization_title=organization_title,
                    email=email,
                    phone=phone,
                    request_counter=request_counter
                )
                if result:
                    contacts_created += 1
                else:
                    contacts_failed += 1

        except Exception as e:
            contacts_failed += 1
            print(f"❌ Row {index+2}: Error processing row: {e}")

    # Print summary with emojis
    print("\n" + "="*60)
    print(" 📊 SYNC SUMMARY")
    print("="*60)
    print(f"🔍 Contacts processed: {contacts_processed}")
    print(f"✅ Contacts updated: {contacts_updated}")
    print(f"➕ Contacts created: {contacts_created}")
    print(f"⏩ Contacts skipped: {contacts_skipped}")
    print(f"❌ Contacts failed: {contacts_failed}")
    print("="*60)

# Main execution block
if __name__ == "__main__":
    from google.colab import files
    import os

    # Clear any existing token to force re-authentication with proper scopes
    if os.path.exists('token.pickle'):
        os.remove('token.pickle')
        print("Removed existing token to ensure proper authentication")

    # Check for service account credentials
    service_account_exists = os.path.exists('service-account.json')
    if not service_account_exists:
        print("Service account credentials not found.")
        print("You can upload a service account JSON file for automated authentication,")
        print("or use OAuth flow which requires manual authorization steps.")

        print("\nDo you want to upload a service account JSON file? (yes/no)")
        upload_choice = input().lower()

        if upload_choice.startswith('y'):
            print("Please upload your service account JSON file (rename it to service-account.json after upload):")
            uploaded = files.upload()

            if len(uploaded) > 0:
                # Rename the first file to service-account.json
                first_file = list(uploaded.keys())[0]
                with open(first_file, 'rb') as f_in:
                    content = f_in.read()

                with open('service-account.json', 'wb') as f_out:
                    f_out.write(content)

                service_account_exists = True
                print("Service account credentials saved as service-account.json")

    # If no service account, check for OAuth credentials
    if not service_account_exists and not os.path.exists('credentials.json'):
        print("Please upload your Google API OAuth credentials file (credentials.json):")
        uploaded = files.upload()

        if 'credentials.json' not in uploaded:
            print("Error: credentials.json file not uploaded. This is required to authenticate with Google API.")
            exit()
        else:
            print("Credentials file uploaded successfully!")
    elif service_account_exists:
        print("Using existing service account credentials")
    else:
        print("Using existing OAuth credentials.json file")

    # Check if CSV file exists or request upload
    csv_filename = None
    if os.path.exists('google_contacts_.csv'):
        csv_filename = 'google_contacts_.csv'
        print(f"Using existing file: {csv_filename}")
    else:
        print("\nPlease upload your CSV file with contact information:")
        uploaded = files.upload()

        if len(uploaded) == 0:
            print("Error: No CSV file uploaded.")
            exit()
        else:
            csv_filename = list(uploaded.keys())[0]
            print(f"Processing file: {csv_filename}")

    # Provide authentication instructions based on method
    if service_account_exists:
        print("\n" + "="*60)
        print(" 🔑 SERVICE ACCOUNT AUTHENTICATION")
        print("="*60)
        print("You will need to provide the email address of the user to impersonate.")
        print("This should be an administrator or user with access to contacts.")
        print("The service account must have domain-wide delegation enabled.")
        print("="*60 + "\n")
    else:
        print("\n" + "="*60)
        print(" 🔑 OAUTH AUTHENTICATION INSTRUCTIONS")
        print("="*60)
        print("You will be prompted to authorize access to your Google Contacts.")
        print("This requires:")
        print("1. Clicking the authorization link that will appear")
        print("2. Selecting your Google account")
        print("3. Clicking 'Continue' when asked for permissions")
        print("4. Copying the authorization code back to this notebook")
        print("="*60 + "\n")

    # Process the uploaded CSV file
    process_contacts_from_csv(csv_filename)